In [36]:
import pandas as pd
import string
import re
import os
from datetime import datetime
import pickle

In [6]:
tweet_column = 'content'

In [8]:
file_number = 0
for (dirname, dirs, files) in os.walk('raw_files'):
        for filename in files:
            file_number += 1
            if filename.endswith('.csv'):
                df_tweets_full = pd.read_csv(os.path.join('raw_files', filename), encoding='utf-8')
            df_english = df_tweets_full[df_tweets_full.language == 'English']
            df_english = df_english[pd.notnull(df_english[tweet_column])]
            unneeded_col = ['external_author_id', 'harvested_date', 'region', 'language', 'publish_date', 'account_type',
                            'alt_external_id', 'article_url', 'following', 'followers', 'updates', 'post_type',
                            'tco1_step1', 'tco2_step1', 'tco3_step1', 'new_june_2018', 'retweet', 'account_category']
            df_english.drop(unneeded_col, axis=1, inplace=True)
            
            if not os.path.exists('unclean_pickles'):
                os.makedirs('unclean_pickles')
            df_english.to_pickle('unclean_pickles/tweets' + str(file_number) + '.pkl')
            print('Wrote File:', str(file_number))

Wrote File: 1
Wrote File: 2
Wrote File: 3
Wrote File: 4
Wrote File: 5
Wrote File: 6


/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Wrote File: 7
Wrote File: 8
Wrote File: 9
Wrote File: 10
Wrote File: 11


/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Wrote File: 12


/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Wrote File: 13


In [9]:
full_df_list = []

for (dirname, dirs, files) in os.walk('unclean_pickles'):
    for filename in files:
        with open(os.path.join('unclean_pickles', filename), 'rb') as f:
            full_df_list.append(pd.read_pickle(f))

In [10]:
full_df = pd.concat(full_df_list, axis=0, ignore_index=True)

In [11]:
full_df.columns

Index(['author', 'content', 'tweet_id'], dtype='object')

In [24]:
mentions = []
for tweet in full_df.content:
    mentions.append(re.findall("(^|[^@\w])@(\w{1,15})", tweet))

In [17]:
def extract_hash_tags(s):
    return set(part[1:] for part in s.split() if part.startswith('#'))

In [20]:
hashtag = []
for tweet in full_df.content:
    hashtag.append(extract_hash_tags(tweet))

In [30]:
full_df['hashtags'] = hashtag
full_df['mentions'] = mentions


In [34]:
full_df.head()

,author,content,tweet_id,hashtags,mentions
0,SEATTLE_POST,Israeli police say Palestinian home in West Ba...,679252328986238977,{news},[]
1,SEATTLE_POST,Former IAAF chief Lamine Diack faces new corru...,679259242541395972,{sports},[]
2,SEATTLE_POST,SpaceX launches rocket 6 months after accident...,679265189372407808,{local},[]
3,SEATTLE_POST,The Latest: Fatalities feared in I-49 crash in...,679275250069078016,{news},[]
4,SEATTLE_POST,Danish “Graceland” museum changes name after l...,679287094682902528,{entertainment},[]


In [37]:
with open('hashtag_mention_df.pickle', 'wb') as file:
    pickle.dump(full_df, file, protocol=pickle.HIGHEST_PROTOCOL)